# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys

repo_path = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.append(repo_path)

In [3]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [4]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

In [ ]:
action = np.random.randint(action_size)

In [ ]:
env_info = env.step(action)[brain_name]

In [ ]:
next_state = env_info.vector_observations[0]

In [ ]:
reward = env_info.rewards[0]

In [ ]:
done = env_info.local_done[0]

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
acc_steps = 0
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    acc_steps += 1
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {} in {} steps {}".format(score, acc_steps))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
from collections import deque
import pandas as pd

def train(env, brain_name, agent, n_episodes=2000, max_t=1000,
        eps_start=1.0, eps_end=0.01, eps_decay=0.995,
        model_save_path='checkpoint.pth', score_solved=13., score_win=100):
    """
    Train DQ-Learning agent on a given environment, based on epsilon-greedy policy and GLIE evolution of epsilon parameter
    When the game is considered solved, save the  DQ-Net underlaying the agent in a given path
    Params
    ======
        env: Environment to solve an episodic game. Should behave like:
            state = env.reset()
            next_state, reward, done, _ = env.step(action)
        agent: DQ-Learning Agent, should estimate and optimal policy estimating Q function using a DQN
        n_episodes (int): Number of episodes to simulate
        max_t (int): Max number of time steps (transitions) on each episode
        eps_start (float): Epsilon parameter starting value (at first episode)
        eps_end (float): Epsilon min value
        eps_decay (float): Epsilon decay rate
        model_save_path (str): Path to persist model
        score_solved (float): Score to consider the game solved
    Returns
    ======
    scores (list): Average reward over 100 episodes
    """

    scores = []  # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start  # initialize epsilon
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]  # get the current state
        score = 0  # initialize the score
        for t in range(max_t):
            action = agent.act(state, eps)

            env_info = env.step(action)[brain_name]  # send the action to the environment
            next_state = env_info.vector_observations[0]  # get the next state
            reward = env_info.rewards[0]  # get the reward
            done = env_info.local_done[0]  # see if episode has finished

            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)  # save most recent score
        scores.append(score)  # save most recent score
        eps = update_epsilon(eps_end, eps_decay, eps)  # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % score_win == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window) >= score_solved:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(
                i_episode - score_win, np.mean(scores_window)))
            agent.save_network(model_save_path)
            break
    env.close()
    
    return pd.Series(index=range(1, len(scores) +1), data=scores, dtype=np.float32, name='score')

def update_epsilon(eps_end, eps_decay, eps_curr):
    """

    """
    return max(eps_end, eps_decay * eps_curr)

In [7]:
from src.dqn_agent import AgentDQ


In [8]:
agent_dq = AgentDQ(state_size=37, action_size=4, gamma=0.99, hidden_layers=[64, 32], drop_p=0.2,
                 batch_size=64, learning_rate=5e-4, soft_upd_param=1e-3, update_every=4, buffer_size=int(1e5), seed=123)


In [9]:
scores_dq = train(env, brain_name=brain_name, agent=agent_dq, n_episodes=2000, max_t=1000, 
        eps_start=1.0, eps_end=0.01, eps_decay=0.995, model_save_path='models/dq_checkpoint_v02.pth')

Episode 100	Average Score: -0.11
Episode 200	Average Score: 1.721
Episode 300	Average Score: 3.41
Episode 400	Average Score: 7.48
Episode 500	Average Score: 10.65
Episode 591	Average Score: 13.02
Environment solved in 491 episodes!	Average Score: 13.02


In [11]:
scores_df = scores_dq.to_frame('score')
scores_df['score_mave100'] = scores_df['score'].rolling(100).mean()
scores_df['experiment'] = 'dqn:v02'
scores_df.index.name = 'idx_episode'

In [12]:
checkpoint_metadata = pd.Series(index=['N_episodes', 'gamma', 'hidden_layers', 'drop_p',
                 'batch_size', 'learning_rate', 'soft_upd_param', 'update_every', 'buffer_size','solved',
                'checkpoint'],
         data = [len(scores_dq), 0.99, [64, 32], 0.2, 64, 5e-4, 1e-3, 4, int(1e5), True, 'dq_checkpoint_v02.pth'], name='experiment:dqn:v02')
checkpoint_metadata

N_episodes                          591
gamma                              0.99
hidden_layers                  [64, 32]
drop_p                              0.2
batch_size                           64
learning_rate                    0.0005
soft_upd_param                    0.001
update_every                          4
buffer_size                      100000
solved                             True
checkpoint        dq_checkpoint_v02.pth
Name: experiment:dqn:v02, dtype: object

In [13]:
import datetime as dt
experiment_dt = dt.datetime.strftime(dt.datetime.now(), "%Y%m%d%H%M%S")

In [14]:
checkpoint_metadata.to_json(f'models/experiments/hparams_{experiment_dt}.json')
scores_df.to_csv(f'models/experiments/scores_{experiment_dt}.csv')